In [ ]:
# t3_optionB_compstat.py — T=3 with a second decision at t=2; t=2 Think grants period-3 exclusivity
# Outputs:
#   figures/t3B_think_vs_r.png
#   figures/t3B_think_vs_c.png
#   figures/t3B_vs_t2_think_vs_r.png
#   results/t3B_summary.csv

import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
# Create output folders (idempotent)
os.makedirs("figures", exist_ok=True); os.makedirs("results", exist_ok=True)

# ----- Baseline parameters (match your paper) -----
# p: prior that LEFT arm is good; s: success prob if the good arm is pulled; V: prize value.
# c0, r0: baseline thinking cost and breakthrough probability used as anchors/dots in plots.
p, s, V = 0.6, 0.7, 1.0
c0, r0  = 0.10, 0.40   # used for dots

def Lambda(p,s,V,r):
    """One-period continuation value Λ for a single remaining period: s*V*[ r + (1-r)*p ]."""
    return s*V*(r + (1-r)*p)

flat = p*s*V  # shorthand for p*lambda*V — expected DO payoff without new information

# ---------- T=2 baseline (for overlay) ----------
def xstar_T2(p,s,V,c,r):
    """Closed-form T=2 mix: returns (x_do, 1-x) for given primitives and (c,r)."""
    L = Lambda(p,s,V,r)
    if (flat + c) <= 0.5*L: return 0.0, 1.0         # Always-Think
    if (flat + c) >= L:     return 1.0, 0.0         # Always-Do
    x = 2.0*(flat + c)/L - 1.0                      # Interior: indifference
    x = float(np.clip(x, 0.0, 1.0))
    return x, 1.0 - x

# ============================================================
# ===============  t=2 subgame (Option B)  ===================
# ============================================================
# Three configurations at t=2 depending on t=1 rights:
#   RH: I have period-2 exclusivity (opponent has none).
#   NE: No exclusivity / shared right (contestable or shared).
# All t=2 payoffs include the period-3 continuation and use the same one-period Λ.

# ---- NE case (contestable/shared), symmetric in players ----
# If I Do and opponent Does with prob x: U_D_NE(x) = flat*(1 - x/2)
# If I Think: U_T_NE(x) = -c + (1 + x)/2 * L
def t2_NE_equilibrium(c, L):
    """
    NE (no exclusivity at t=2). If I Do, my total (period-2 + period-3) equals 'flat'
    regardless of opponent's mix. If I Think, I get -c + 0.5*(1+x)*L.
    Corners match the familiar T=2 thresholds.
    Returns: (x_NE, think_freq, U_NE, region), where:
      x_NE      = equilibrium prob of Do at t=2 under NE,
      think_freq= 1 - x_NE,
      U_NE      = my value at the NE equilibrium,
      region    = {'always-think','always-do','interior'}.
    """
    # Always-Think if even at x=0 (worst for Think) we prefer T: -c + 0.5L >= flat
    if (-c + 0.5*L) >= flat:
        return 0.0, 1.0, (-c + 0.5*L), "always-think"
    # Always-Do if even at x=1 (best for Think) T underperforms Do: -c + L <= flat
    if (-c + L) <= flat:
        return 1.0, 0.0, flat, "always-do"
    # Interior mix solves flat = -c + 0.5*(1+x)*L  ->  x = 2*(flat+c)/L - 1
    x = float(np.clip(2.0*(flat + c)/L - 1.0, 0.0, 1.0))
    return x, 1.0 - x, flat, "interior"


# ---- RH vs NRH case (I have period-2 exclusivity) ----------
# RH: U_D_RH = flat   (I capture period-2 prize regardless of opponent)
#     U_T_RH(x_NRH) = -c + (1 + x_NRH)/2 * L
# NRH: U_D_NRH = 0    (cannot appropriate under exclusivity, but may share p3 if both Do)
#      U_T_NRH(x_RH) = -c + (1 + x_RH)/2 * L
# Best-response structure implies two main regimes with a low-cost corner where both Think.
def t2_RH_equilibrium(c, L):
    """
    RH vs NRH at t=2 (I am the right-holder). Evaluate pure-strategy Nash via BRs.
    Returns: (x_RH, x_NRH, U_RH, U_NRH, region), where x_* are Do probabilities.
    Threshold reminders:
      RH's BR to NRH=Think: Do if c >= 0.5L - flat (else Think)
      RH's BR to NRH=Do:    Do if c >= L - 1.5*flat (else Think)
      NRH's BR to RH=Do:    Think if c <= L - 0.5*flat (else Do)
      NRH's BR to RH=Think: Think if c <= 0.5L        (else Do)
    """
    # Candidate 1: (RH Do, NRH Think) — common regime when costs are moderate
    cond1 = (c >= 0.5*L - flat) and (c <= L - 0.5*flat)
    if cond1:
        x_RH, x_NRH = 1.0, 0.0
        U_RH  = flat                           # p2 exclusive secured; no p3 bump if NRH thinks
        U_NRH = -c + L                         # unique T -> p3 exclusive
        return x_RH, x_NRH, U_RH, U_NRH, "RH-do_NR-think"

    # Candidate 2: (RH Do, NRH Do) — both exploit; NRH can only share p3
    cond2 = (c >= L - 1.5*flat) and (c >= L - 0.5*flat)
    if cond2:
        x_RH, x_NRH = 1.0, 1.0
        U_RH  = flat + 0.5*flat               # p2 exclusive + p3 contestable
        U_NRH = 0.5*flat                      # p3 contestable share
        return x_RH, x_NRH, U_RH, U_NRH, "both-do"

    # Candidate 3: (RH Think, NRH Think) — very low c; both invest
    cond3 = (c <= 0.5*L - flat) and (c <= 0.5*L)
    if cond3:
        x_RH, x_NRH = 0.0, 0.0
        U_RH  = -c + 0.5*L
        U_NRH = -c + 0.5*L
        return x_RH, x_NRH, U_RH, U_NRH, "both-think"

    # Otherwise, select between the two Do-at-RH regimes by NRH's best reply when RH=Do
    if (-c + L) >= 0.5*flat:
        # NRH prefers Think given RH=Do
        x_RH, x_NRH = 1.0, 0.0
        U_RH, U_NRH = flat, (-c + L)
        reg = "RH-do_NR-think"
    else:
        # NRH prefers Do given RH=Do
        x_RH, x_NRH = 1.0, 1.0
        U_RH, U_NRH = flat + 0.5*flat, 0.5*flat
        reg = "both-do"
    return x_RH, x_NRH, U_RH, U_NRH, reg


# ============================================================
# ===============  t=1 equilibrium (Option B)  ===============
# ============================================================
# Let q be opponent's T prob at t=1 (so opponent's D prob x1 = 1 - q).
# Period-1 immediate payoff if I Do: flat * [ q*1 + (1-q)*1/2 ] = flat*(1 - q/2)
# If I Think: I pay -c and forgo period-1 exploit.
# Future values hinge on who gets period-2 right and the t=2 subgame outcome:
#   If I Do:
#       with prob q     -> opponent T -> I am NRH at t=2 -> value = U_NRH
#       with prob 1 - q -> opponent D -> NE at t=2       -> value = U_NE
#   If I Think:
#       with prob q     -> both T     -> NE at t=2       -> value = U_NE
#       with prob 1 - q -> opp D      -> I am RH at t=2  -> value = U_RH
def t1_equilibrium_optionB(p,s,V,c,r):
    """Solve the symmetric t=1 best response via corner checks then bisection for the interior."""
    L = Lambda(p,s,V,r)
    # t=2 stage values under NE and under asymmetric RH/NRH
    x_NE, tf_NE, U_NE, _ = t2_NE_equilibrium(c, L)
    x_RH, x_NRH, U_RH, U_NRH, _ = t2_RH_equilibrium(c, L)

    # Expected payoffs at t=1 as functions of opponent's T prob q
    def UD(q):
        return flat*(1 - q/2) + q*U_NRH + (1 - q)*U_NE
    def UT(q):
        return -c + q*U_NE + (1 - q)*U_RH

    # Corner checks: best replies to opponent D (q=0) and opponent T (q=1)
    if UD(0.0) <= UT(0.0):  # Best reply to D is T -> Always-Think
        return 0.0, 1.0, "always-think", L, (x_NE, U_NE, x_RH, x_NRH, U_RH, U_NRH)
    if UD(1.0) >= UT(1.0):  # Best reply to T is D -> Always-Do
        return 1.0, 0.0, "always-do", L, (x_NE, U_NE, x_RH, x_NRH, U_RH, U_NRH)

    # Interior case: solve UD(q) = UT(q) by bisection (continuous, monotone difference)
    lo, hi = 0.0, 1.0
    for _ in range(80):
        mid = 0.5*(lo+hi)
        f = UD(mid) - UT(mid)
        if f == 0 or (hi-lo) < 1e-10:
            q_star = mid; break
        f_lo = UD(lo) - UT(lo)
        if np.sign(f) == np.sign(f_lo): lo = mid
        else: hi = mid
    q_star = 0.5*(lo+hi)
    # Return (x_do, think_freq, region, L, details from t=2)
    return 1.0 - q_star, q_star, "interior", L, (x_NE, U_NE, x_RH, x_NRH, U_RH, U_NRH)

# ============================================================
# =====================  PLOTTING  ===========================
# ------------------------------------------------------------

def plot_vs_r():
    """Overlay T=3 Option B vs T=2 baseline as r varies (c fixed at c0)."""
    r_grid = np.linspace(0.01, 0.99, 300)
    tf_B  = []
    tf_T2 = []
    for r in r_grid:
        x_do, tf, _, _, _ = t1_equilibrium_optionB(p,s,V,c0,r)
        tf_B.append(tf)
        tf_T2.append(xstar_T2(p,s,V,c0,r)[1])

    fig, ax = plt.subplots(figsize=(7.6,4.4), constrained_layout=True)
    ax.plot(r_grid, tf_T2, label="T=2 (baseline)")
    ax.plot(r_grid, tf_B,  "--", label="T=3 (Option B)")
    # Baseline point at r0: dot for T=2, x-marker for Option B
    tf0_B = t1_equilibrium_optionB(p,s,V,c0,r0)[1]
    tf0_T2 = xstar_T2(p,s,V,c0,r0)[1]
    ax.scatter([r0],[tf0_T2], color="k", zorder=3)
    ax.scatter([r0],[tf0_B],  color="k", marker="x", zorder=3)
    ax.set_xlabel("Breakthrough probability (r)")
    ax.set_ylabel("Think frequency at t=1 (1 - x*)")
    ax.set_title("Period-1 Think: T=2 baseline vs T=3 (Option B)")
    ax.legend()
    fig.savefig("figures/t3B_vs_t2_think_vs_r.png", dpi=200, bbox_inches="tight")
    plt.close(fig)

def plot_vs_c():
    """Option B Think frequency as c varies (r fixed at r0)."""
    c_grid = np.linspace(0.0, 0.8, 300)
    tf_B = []
    for c in c_grid:
        tf_B.append(t1_equilibrium_optionB(p,s,V,c,r0)[1])

    fig, ax = plt.subplots(figsize=(7.6,4.4), constrained_layout=True)
    ax.plot(c_grid, tf_B, label="T=3 (Option B)")
    ax.scatter([c0],[t1_equilibrium_optionB(p,s,V,c0,r0)[1]], color="k", zorder=3)  # baseline dot
    # Cosmetic only: show monotone decline in the interior region visually
    ax.set_xlabel("Thinking cost (c)")
    ax.set_ylabel("Think frequency at t=1 (1 - x*)")
    ax.set_title("T=3 (Option B): Think frequency vs thinking cost (r fixed)")
    ax.legend()
    fig.savefig("figures/t3B_think_vs_c.png", dpi=200, bbox_inches="tight")
    plt.close(fig)

def plot_vs_r_optionB_only():
    """Option B Think frequency vs r (no T=2 overlay), c fixed at c0."""
    r_grid = np.linspace(0.01, 0.99, 300)
    tf_B = [t1_equilibrium_optionB(p,s,V,c0,r)[1] for r in r_grid]
    fig, ax = plt.subplots(figsize=(7.6,4.4), constrained_layout=True)
    ax.plot(r_grid, tf_B, label="T=3 (Option B)")
    ax.scatter([r0],[t1_equilibrium_optionB(p,s,V,c0,r0)[1]], color="k", zorder=3)  # baseline dot
    ax.set_xlabel("Breakthrough probability (r)")
    ax.set_ylabel("Think frequency at t=1 (1 - x*)")
    ax.set_title("T=3 (Option B): Think frequency vs breakthrough probability (c fixed)")
    ax.legend()
    fig.savefig("figures/t3B_think_vs_r.png", dpi=200, bbox_inches="tight")
    plt.close(fig)

# Run all plots
plot_vs_r()
plot_vs_c()
plot_vs_r_optionB_only()

# Small summary CSV at the baseline point (collect t=1 and t=2 subgame details for reference)
x_do, tf, region, L, (xNE, U_NE, xRH, xNRH, U_RH, U_NRH) = t1_equilibrium_optionB(p,s,V,c0,r0)
pd.DataFrame([{
    "p":p,"s":s,"V":V,"c":c0,"r":r0,"Lambda":L,
    "t1_x_do":x_do,"t1_think_freq":tf,"t1_region":region,
    "t2_NE_x_do":xNE,"t2_NE_value":U_NE,
    "t2_RH_x_do":xRH,"t2_NRH_x_do":xNRH,"t2_RH_value":U_RH,"t2_NRH_value":U_NRH
}]).to_csv("results/t3B_summary.csv", index=False)

print("Saved: figures/t3B_think_vs_r.png, figures/t3B_think_vs_c.png, figures/t3B_vs_t2_think_vs_r.png, results/t3B_summary.csv")


Saved: figures/t3B_think_vs_r.png, figures/t3B_think_vs_c.png, figures/t3B_vs_t2_think_vs_r.png, results/t3B_summary.csv
